In [122]:
import nltk
import os 

from nltk.tag import UnigramTagger
from nltk.corpus import treebank
from nltk.tokenize import TreebankWordTokenizer
import tarfile
from nltk.corpus import sentiwordnet as swn


#nltk.download()

In [5]:
train_sents = treebank.tagged_sents()[:3000]
tagger = UnigramTagger(train_sents)

In [18]:
tagger.tag(treebank.sents()[101])

[('A', 'DT'),
 ('full', 'JJ'),
 (',', ','),
 ('four-color', 'JJ'),
 ('page', 'NN'),
 ('in', 'IN'),
 ('Newsweek', 'NNP'),
 ('will', 'MD'),
 ('cost', 'NN'),
 ('$', '$'),
 ('100,980', 'CD'),
 ('*U*', '-NONE-'),
 ('.', '.')]

In [19]:
tf = tarfile.open("review_polarity.tar")
#tf.extractall()

In [95]:
tf = tarfile.open("wn3.1.dict.tar")
tf.extractall()

In [87]:
def init_ad_review(pos_neg):
    list_rev = []
    for x in os.listdir('txt_sentoken/'+pos_neg+'/'):
        with open('txt_sentoken/'+pos_neg+'/'+x,'r') as f:
            file = f.read()
            list_rev.append(file)
    #split sentence by word
    tokenizer = TreebankWordTokenizer()

    list_rev_w_tag = []
    for review in list_rev:
        list_rev_w_tag.append(tagger.tag(tokenizer.tokenize(review)))


    list_rev_w_tag_RB = []
    sentence = []
    for review in list_rev_w_tag:
        for word in review:
            if not word[1] is None:
                if 'RB' in word[1]:
                    sentence.append(word)
        list_rev_w_tag_RB.append(sentence)
        sentence = [] 
    return list_rev_w_tag_RB

In [88]:
list_pos_w_tag_RB = init_ad_review('pos')

In [89]:
list_neg_w_tag_RB = init_ad_review('neg')

In [141]:
def identify_pos_neg(review):
    pos = 0
    neg = 0
    for word in review:
        try:
            pos +=swn.senti_synset('{word}.r.1'.format(word = word[0])).pos_score()
            neg +=swn.senti_synset('{word}.r.1'.format(word = word[0])).neg_score()
        except:
            pass
    return pos, neg

In [148]:
count = 0
for review in list_pos_w_tag_RB:
    pos , neg  = identify_pos_neg(review)
    if pos > neg:
        count +=1